In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from pandas import DataFrame as df
import os
import sys
import datetime

sys.path.append(os.path.abspath("../"))
from shibaer import util, features, visualization
pd.options.display.float_format = "{:.2f}".format

tprint = lambda s: print('\n--- %s ---' %s)
pprint=lambda i: '%i (%.2f)' %(i,float(i)/len(fraud_events))
hour_diff = lambda i:i.total_seconds()/360 if type(i) is not None else i


    
thumbdrive = 'DATAG'
data_dir = 'ER'
data = util.load_pickle_files(thumbdrive, data_dir, is_small=False)
data_with_targets = features.add_death_columns(data)
target_variables = [d for d in data_with_targets.columns if 'T_' in d]
target_variables

meta_data=util.read_metadata()
numeric_cols = meta_data.loc[meta_data.data_type == 'numeric'].index


Found the following pickle files:  ['2013 PROC 4.8.18.pkl', '2014 PROC 4.8.18.pkl', '2015 PROC 4.8.18.pkl', '2016 PROC 4.8.18.pkl', '2017 PROC 4.8.18.pkl']
Loading : 2013 PROC 4.8.18.pkl
Loading : 2014 PROC 4.8.18.pkl
Loading : 2015 PROC 4.8.18.pkl
Loading : 2016 PROC 4.8.18.pkl
Loading : 2017 PROC 4.8.18.pkl
Finished loading data
Total number of rows:  595673


## Remove outliers from nuerical variables

In [ ]:
data = util.preprocess_data(data, numeric_cols= numeric_cols, percentiles=[0.001,0.999])

##  Background variables mortality exploration

### Percent of population with background variable and percent mortality per background

In [ ]:
target_columns=['T_is_dead', 'T_release_date', 'T_total_time_hospital',
       'T_mortality_ER', 'T_mortality_hospitalization',
       'T_mortality_after_hospitalization', 'T_mortality2d', 'T_mortality30d',
       'T_mortality60d', 'T_mortality_type']

#ndata= util.convert_to_numeric(data, numeric_cols=numeric_cols)
print('..')
data_with_target_cols=features.add_death_columns(data.head(100))
print('..')
background_var= 'background_diagnoses'
s= data[background_var]
bg_diagnosis=df(pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0), index=data.index)
print('saving to csv...')
bg_diagnosis.to_csv(background_var+'.csv')
#(bg_diagnosis.sum()/len(bg_diagnosis)).plot(kind='bar', 

In [ ]:
print('saving to csv...')
bg_diagnosis.to_csv(background_var+'head1000.csv')
bg_diagnosis.loc[bg_dianosis.sum()>10]

In [ ]:
#background_var= 'background_diagnoses'
target_col= 'T_mortality30d'
head=50
bg_diagnosis_with_target=pd.concat([bg_diagnosis,data_with_target_cols[target_col]],axis=1)

target_per_bckg= bg_diagnosis_with_target.groupby(target_col).sum()
a.index


total_dead_per_bckg= (target_per_bckg.loc[True]).sort_values(ascending=False)
top50= total_dead_per_bckg.index[:49] 
top10= total_dead_per_bckg.index[:9] 
tprint('top 50 %s for %s (based on total number of deaths):' %(background_var,target_col))
print(top50)

percent_dead_per_bckg= (target_per_bckg.loc[True]/bg_diagnosis.sum()).loc[total_dead_per_bckg.index]



In [ ]:
#total_dead_per_bckg.head(head).plot.bar(figsize=(20,10), title='Total number %s per %s' %(target_col, background_var))
target_per_bckg.T.loc[total_dead_per_bckg.index].head(head).plot.bar(stacked=True,figsize=(20,5),
                                            title='Total %s per %s' %(target_col, background_var))

#a.loc[False]/a.loc['True']
#.groupby(target_col).sum().T.stackplot(figsize=(20,10))
#print(targets)
#pd.DataFrame.from_items(zip(l.index, l.values))

In [ ]:
percent_dead_per_bckg.loc[total_dead_per_bckg.index].head(head).plot.bar(figsize=(20,5), title='Percent %s per %s' %(target_col, background_var))

### pie plot percent of background variable per target variable

In [ ]:
target_per_bckg[top10].T.plot(kind='pie', subplots=True,figsize=(15,7),autopct='%1.1f%%',title='Percent %s per %s' %(background_var,target_col))
#target_per_bckg[top10].plot.bar()

### Percent of background variable per different target variables

In [ ]:
target_vars = ['T_mortality2d', 'T_mortality30d','T_mortality60d']
bg_diagnosis_with_all_targets=pd.concat([bg_diagnosis,data_with_target_cols[target_vars]],axis=1)
'''full_targets = df(columns=target_vars)
full_targets_percent = df(columns=target_vars)

for t in target_vars:
    tt=bg_diagnosis_with_all_targets.groupby(t).sum()
    full_targets[t]= tt.T[True]'''
    
full_targets_percent = full_targets/full_targets.sum()
#full_targets_percent.drop(target_vars[1:], inplace=True)
#full_targets.drop(target_vars[1:], inplace=True)
#print(full_targets)
full_targets['total'] = full_targets.sum(axis=1)
full_targets.sort_values(by='total',ascending=False,inplace=True)
top= full_targets.head(10).index

full_targets[target_vars].loc[top].T.plot.bar(stacked=True,figsize=(20,5))#(kind='bar')
full_targets_percent[target_vars].loc[top].T.plot(figsize=(20,5))#(kind='bar')
full_targets[target_vars].loc[top].plot(kind='pie', subplots=True,figsize=(20,6),autopct='%1.1f%%',
                                       title='Percent %s per %s' %(background_var,target_col))
print(top)
#full_targets[target_vars].loc[top].plot.bar(figsize=(20,5))
#full_targets.drop(target_vars,axis=1, inplace=True)

#targets_per_bckg= bg_diagnosis_with_all_target.groupby(target_vars).sum()
    #print(targets_per_bckg)
#['T_mortality_within']
#target_per_bckg[top10].T.plot(kind='pie', subplots=True,figsize=(15,7),autopct='%1.1f%%',

In [ ]:
# Visualization funcs
import datetime

def stack_plot(data, index_axis, stack_var='age',title=None, ylim=None):
    if title is None:
        title=stack_var +' by ' + index_axis.dtype
    data[stack_var].groupby(index_axis).value_counts().unstack().plot(kind='bar', 
                                                                      stacked=True, figsize=(20,5), title=title, ylim=ylim)
    
def plot_with_legend(data,group_var='gender', plot_var='age_on_date',drop_values=[]):
    fig, ax = plt.subplots()
    groups = data.groupby(group_var)[plot_var]
    for k, v in groups:
        if k not in drop_values:
            v.hist(label=k, alpha=.75, ax=ax, bins = 40)
    ax.legend()

def plot_time_of_death_from(time_from='admission_date_min', drop_negatives=True, as_time='D', max_period= 1200):
    death_delta= (death_data.death_date - death_data[time_from])/np.timedelta64(1,as_time)
    if drop_negatives:
        death_delta = death_delta.loc[death_delta>0] #remove negative death deltas
    death_delta.loc[death_delta<=max_period].hist(bins=100)
    #death_days_from_admission.hist()
    tprint('Relative time of death after %s (uniqe id_coded) %i-%s '%(time_from,max_period,as_time))
    return death_delta

def remove_outliers(data, high=0.99, low=0.01):
    quant_df = data.quantile([low, high])
    print(quant_df)
    for name in list(quant_df.columns):
        #if quant_df.columns:
            data = data[(data[name] >= quant_df.loc[low, name]) & (data[name] < quant_df.loc[high, name])]
    return data

In [ ]:
death_data = data_with_targets.loc[data_with_targets.T_is_dead]
print(len(death_data))
death_data = death_data.sort_values('admission_date_min').drop_duplicates(subset =['id_coded'], keep='last')

print(len(death_data))

#dd=plot_time_of_death_from(time_from='T_release_date',max_period=48, as_time='h')
data.columns
percent_missing= data.count()/len(data)
tprint('percent of non-missing values (<90%)')
percent_missing.loc[percent_missing<0.9].plot(kind='bar',figsize=(20,5))

In [ ]:
d=data_with_targets
plot_events_by_time(d.loc[d.T_mortality2d], by_time='admission_date_min',stack_var = 'esi_chameleon')

death_data.groupby(['T_mortality2d'])['esi_chameleon'].plot(kind='bar')
#death_data.esi_chameleon.value_counts()
#death_data.groupby().count()['gender'].plot(kind='bar')
#stack_var
#death_data.head()

In [ ]:
target_cols= ['pulse']
group_by_col = 'T_is_dead'
#clean_data = data_with_targets.loc[datw_with_targets]\
def smart_boxplot(data, filter_var = 'is_hospitalization', remove_outliers = True, group_by_col='T_is_dead', target_cols=['pulse','CPK-MB']):
    data=data.copy()
    data_col=data
    #data.reset_index(inplace=True)
    for t in target_cols:     
        tcol=data[t]
        #tcol=tcol.dropna()
        print(tcol.dtype)
        if remove_outliers:
            tcol= tcol.map({'<10':0})
            q=tcol.dropna().astype(float).quantile(0.99)
            data_col= data_col.loc[tcol <= q]
            data_col.head()
            #print(tcol.dtype)
            #print((tcol>tcol.quantile(0.99)).value_counts())
            #col_ind = remove_outliers(tcol).index
             # all columns including t
        data_col.loc[data_col[filter_var]].boxplot(by=group_by_col,column=t)
        
ax = sns.boxplot(x="gender", y="pulse", hue="T_is_dead",
                 data=data_with_targets, palette="Set3")
#data_with_targets.groupby(group_by_col).describe().T

#plot_with_legend(death_data)
#plot_with_legend(data_with_targets, group_var='T_is_dead', plot_var='age_on_date',drop_values=[] )
#data_col.
#smart_boxplot(data_with_targets)



## Descriptive Statistics

In [ ]:
md= pd.read_csv(os.path.abspath("../docs/ER/meta_data.csv"))
md= md.loc[md.is_load==1]

data_timestamp_index = 'rss_timestamp'
#print(md.head())
print('\n---- ER data categories : ------')
print(md.groupby('category').nunique()['column_name'])
print('\n---- When was the  data gathered : ------')
print(md.groupby('when (b=before, a=after)').nunique()['column_name'])

demographic_cols = md.loc[md.category == 'demographic']['column_name']\
demographic_data = data[demographic_cols]
lab_cols = md.loc[md.category == 'Labs']['column_name']
tprint('demographic details (all)')

pd.options.display.float_format = "{:.2f}".format


def plot_with_legend(data,group_var='gender', plot_var='age_on_date',drop_values=[]):
    fig, ax = plt.subplots()
    groups = data.groupby(group_var)[plot_var]
    for k, v in groups:
        if k not in drop_values:
            v.hist(label=k, alpha=.75, ax=ax, bins = 40)
    ax.legend()
#demographic_data.groupby('gender').plot(kind='bar')
#[])

#demographic_data['gender'].value_counts())
print(lab_cols)

## where were people commited?

In [ ]:
tprint('commited units:')
print(fraud_events.fraud_type.value_counts())    
    
nunique=fraud_events.nunique()
ncount=fraud_events.count()
nnunique=pd.concat([ncount,nunique],axis=1)
nnunique.columns=[ 'count','unique vals']
tprint('Number of values in each fraud label column:')
print(nnunique.loc[label_cols].applymap(pprint))#.applymap(pprint))
tprint('Number of values in each rss_data column:')
print(nnunique.loc[relevant_data_cols].applymap(pprint))

f

In [ ]:
death_data=data_with_targets[data_with_targets.T_is_dead==True]
death_data.